## Exploratory Data Analysis
The purpose of this notebook is to explore the database that we have set up with our earlier scripts. Through our exploration, we hope to understand the business better and create a dashboard that would be meaningful for the business. 

In [1]:
## Loading of relevant packages and database
import pandas as pd
import numpy as np
import pickle
import sqlalchemy as sal

from os import chdir
from tqdm import tqdm

In [2]:
## Establish connection
conn = sal.create_engine("mssql+pyodbc://CHESTERIZA\SQLEXPRESS/cdg?driver=SQL Server?Trusted_Connection=yes")



We will load the dataset and print out the info to identify missing entries and their respective variable data types.

In [3]:
## Loading of database into DataFrame
FactStore_df = pd.read_sql("SELECT * FROM FactStore", conn)
DimStores_df = pd.read_sql("SELECT * FROM DimStores", conn)
DimDates_df = pd.read_sql("SELECT * FROM DimDates", conn)
DimProducts_df = pd.read_sql("SELECT * FROM DimProducts", conn)

In [4]:
FactStore_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1547896 entries, 0 to 1547895
Data columns (total 8 columns):
DateID           1547896 non-null object
StoreID          1547896 non-null int64
ProductID        1547896 non-null int64
OnHandQty        1547896 non-null int64
OnOrderQty       1547896 non-null int64
DaysInStock      1547896 non-null int64
MinDayInStock    1547896 non-null int64
MaxDayInStock    1547896 non-null int64
dtypes: int64(7), object(1)
memory usage: 94.5+ MB


In [5]:
DimStores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 7 columns):
StoreID              305 non-null int64
StoreName            305 non-null object
CityName             305 non-null object
StateProvinceName    305 non-null object
RegionCountryName    305 non-null object
StoreManager         305 non-null object
StoreType            305 non-null object
dtypes: int64(1), object(6)
memory usage: 16.8+ KB


In [6]:
DimDates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 11 columns):
DateID                365 non-null datetime64[ns]
FullDateLabel         365 non-null object
CalendarYear          365 non-null int64
CalendarQuarter       365 non-null int64
CalendarMonth         365 non-null int64
CalendarMonthName     365 non-null object
CalendarDayOfWeek     365 non-null int64
WeekDayName           365 non-null object
EuropeSeason          365 non-null object
NorthAmericaSeason    365 non-null object
AsiaSeason            365 non-null object
dtypes: datetime64[ns](1), int64(4), object(6)
memory usage: 31.5+ KB


In [7]:
DimProducts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 6 columns):
ProductID                 2517 non-null int64
ProductName               2517 non-null object
ProductDescription        2516 non-null object
ProductCategoryName       2517 non-null object
ProductSubcategoryName    2517 non-null object
Manufacturer              2517 non-null object
dtypes: int64(1), object(5)
memory usage: 118.1+ KB


### General Understanding of the Business
Assuming that each store has a unique StoreID, the business currently operates 306 stores in 35 countries and sells 2508 products. 



In [8]:
## General Business Understanding
print("Business has " + str(len(FactStore_df.StoreID.unique())) + " stores in total")
## 

print("Business operates in " + str(len(DimStores_df.RegionCountryName.unique())) + " country")
DimStores_df.RegionCountryName.unique()

print("Business has " + str(len(FactStore_df.ProductID.unique())) + " products in total")


Business has 306 stores in total
Business operates in 35 country
Business has 2508 products in total


We also notice that there is a missing StoreID in the DimStore table. The StoreID 227 is missing from the DimStore Table.

In [9]:
[i for i in FactStore_df.StoreID.unique() if i not in DimStores_df.StoreID.unique()]

[227]

In [10]:
## Type of stores that we operate
print("Type of stores that we operate")
DimStores_df.StoreType.unique()

Type of stores that we operate


array(['Store', 'Reseller', 'Online', 'Catalog'], dtype=object)

In [11]:
print("Product Categories that we carry")
DimProducts_df.ProductCategoryName.unique()

Product Categories that we carry


array(['TV and Video', 'Computers', 'Audio', 'Cameras and camcorders ',
       'Cell phones', 'Music, Movies and Audio Books', 'Games and Toys',
       'Home Appliances'], dtype=object)

In [12]:
## Type of Seasons in the respective countries
print("Europe Seasons")
print(DimDates_df.EuropeSeason.unique())

print("Asia Seasons")
print(DimDates_df.AsiaSeason.unique())

print("North America Season")
print(DimDates_df.AsiaSeason.unique())

Europe Seasons
['Holiday' 'Spring/Business' 'None' 'Back to School']
Asia Seasons
['Holiday' 'Spring/Back to School' 'Tax Time/Summer' 'None']
North America Season
['Holiday' 'Spring/Back to School' 'Tax Time/Summer' 'None']


### Using PowerBI to understand the business further
The objective of the dashboard is to enable stakeholders to make informed decisions about inventory management. 

* <strong>What is the stock volume trendline for respective product categories in the past 12 months?</strong> </br>

Stock volume should not have too much fluctuation. Any anomaly changes in stock volume for each store could indicate operational problems leading to excess inventories or opportunities from surged in consumer demand.

* <strong>How popular are our manufacturers?</strong> </br>

As we are in the business of selling goods, it is important for us to know if our customers have any preference towards any manufacturers. This will help us in our management of supplier relationship and demand forecasting.

* <strong>Which are the top performing product in each market and overall?</strong></br>

As we have so many products, it is important for us to understand which products are doing. 

* <strong>How many stores do we have in each market and what type of stores?</strong></br>

Given that the business is international, it would be important to know where are the cities we operates and how many stores we have there.

### Deciding on the key metrics to use to understand above mentioned questions.

It is not very clear what the variables DaysInStock, MinDayInStock and MaxDayInStock means. Thus, we will make an assumption about the variables.

* OnHand Quantity refers to the goods that the store currently has.
* OnOrder Quantity refers to the goods that have been ordered by customers.
* DaysInStock refers to the average number of days the product has been held by the store before sale.
* MinDayInStock refers to the minimum number of days the product has been held by the store before sale.
* MaxDayInStock refers to the maximum number of days the product has been held by the store before sale.

Success of a retail business often depends on how inventory is being managed. Thus, we will focus on OnHand and OnOrder Quantity for starters. The histogram of the OnHand and OnOrder Quantity reveal a situation where there are several store outliers with significantly higher OnHand and OnOrder Quantity. This means that using an average as an indicator for business metrics would not be desirable.


In [13]:
## To explore time series plot
# test = FactStore_df.merge(DimStores_df, how="inner", on="StoreID")
# test.DateID = test.DateID.apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
# test = test.set_index('DateID')
# test.resample("M").sum().OnOrderQty.plot()

### Feature Engineering (Get Location Coordinates)
We will use Bing Map API to obtain the coordinates of the Stores based on their location names. We will obtain coordinates up to the state province level. If we want to get up to the city level, we will use the city name as the locality parameter value in the Bing Map API. After obtaining the lat and long of the locations, we will create a new table with the location.

In [14]:
import requests

def get_location_bing(countryRegion, adminDistrict, BingMapsKey):
    url = f"http://dev.virtualearth.net/REST/v1/Locations?countryRegion={countryRegion}&adminDistrict={adminDistrict}&key={BingMapsKey}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        location = data['resourceSets'][0]['resources']
        if len(location) > 0:
            location = location[0]
            location = location['geocodePoints'][0]['coordinates']
        else:
            location = None
    return location

In [15]:
BingMapsKey = "" 
location_list = []
for ind, row in tqdm(DimStores_df.iterrows()):
    StoreID = row.StoreID
    countryRegion = row.RegionCountryName
    adminDistrict = row.StateProvinceName
    location = get_location_bing(countryRegion, adminDistrict, BingMapsKey)
    location_xy = {"StoreID": StoreID,
                   "RegionCountryName":countryRegion,
                   "StateProvinceName":adminDistrict,
                   "lat":location[0], 
                   "long":location[1]}
    location_list.append(location_xy)

305it [00:59,  5.10it/s]


In [17]:
# # Saving the file just in case
location_df = pd.DataFrame(location_list)
location_df.to_csv("location.csv")
location_df.head(5)

,StoreID,RegionCountryName,StateProvinceName,lat,long
0,4,United States,Washington,47.411373,-120.556366
1,156,United Kingdom,England,51.500153,-0.126236
2,88,United States,Texas,31.463793,-99.333275
3,214,United Kingdom,England,51.500153,-0.126236
4,201,United Kingdom,England,51.500153,-0.126236


In [18]:
## Writing a location table to act as look up for coordinates
location_df.to_sql("DimStores_Location", conn, if_exists="replace", index=False)